## Citations:
## Images dataset: https://www.kaggle.com/datasets/bryanb/abstract-art-gallery - download the images and place them in the images folder
## According to the submitter, the images were scraped from: https://github.com/cs-chan/ArtGAN/tree/master/WikiArt%20Dataset

In [14]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [15]:
# We have a set of images to train our GAN on. They need to be normalized
image_dir = './images'
image_files = os.listdir(image_dir)
images = []

for image_file in image_files:
    image_path = os.path.join(image_dir, image_file)
    img = load_img(image_path, target_size=(64, 64))
    img_array = img_to_array(img)
    img_array /= 255.0
    images.append(img_array)
x_train = np.array(images)
print(np.shape(img_array[0]))

(64, 3)


In [16]:
# We need a latent dimension to add noise to the generators data
latent_dim = 100

# Define the Generator
generator = Sequential([
    Dense(256, input_dim=latent_dim, activation='relu'),
    Dense(512, activation='relu'),
    Dense(3 * 64 * 64, activation='sigmoid'),
    Reshape((64, 64, 3))
    ]
)

# Define the Discriminator
discriminator = Sequential([
    Flatten(input_shape=(64, 64, 3)),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(1, activation='sigmoid')
    ]
)

In [17]:
#Compile the discriminator and then freeze it
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False

In [18]:
# Build the generator/discriminator model and compile it
gan = Sequential([generator, discriminator])
gan.compile(optimizer='adam', loss='binary_crossentropy')

In [19]:
# Set our hyperparameters
batch_size = 128
epochs = 100
steps_per_epoch = len(x_train) // batch_size

In [ ]:
for epoch in range(epochs):
    for step in range(steps_per_epoch):
        # Train discriminator on real images
        real_images = x_train[np.random.randint(0, len(x_train), batch_size)] #Randomly select 'batch_size' number of images from the training data 
        real_labels = np.ones((batch_size, 1)) #There are real images, set the labels for each image to 1
        d_loss_real = discriminator.train_on_batch(real_images, real_labels) # Training the discriminator so that it will be well equipped to handle the generator images

        # Generate fake images and train discriminator on them
        noise = np.random.normal(0, 1, (batch_size, latent_dim)) #We need some noise to create our fake data
        fake_images = generator.predict(noise) #Generate fake images
        fake_labels = np.zeros((batch_size, 1)) #Indicate that these are fake images by lableing each with a zero label
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels) #Training the discriminator on the fake images so that it wil know the difference

        #Now we have trained the discriminator on real and fake images. Now we train our generator

        # Train the generator (via the GAN model)
        noise = np.random.normal(0, 1, (batch_size, latent_dim)) #Noise vectors for the generated samples
        valid_labels = np.ones((batch_size, 1)) #Labels to make the fake data appear real to the discriminator
        g_loss = gan.train_on_batch(noise, valid_labels) #Treain using the generator/discriminator model

    print(f"Epoch {epoch + 1}/{epochs}, D Loss Real: {d_loss_real[0]}, D Loss Fake: {d_loss_fake[0]}, G Loss: {g_loss}")
